# Build Predictor (XGBoost) Model

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/inference|structured|realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

---

We demonstrate building a ML inference application to predict the rings of Abalone.

After the model is hosted for inference, the payload will be sent as a raw (untransformed) csv string to a real-time endpoint.
The raw payload is first received by the preprocessor container. The raw payload is then transformed (feature-engineering) by the preprocessor, and the transformed record (float values) are returned as a csv string by the preprocessor container.

The transformed record is then passed to the predictor container (XGBoost model). The predictor then converts the transformed record into [`XGBoost DMatrix`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.DMatrix) format, loads the model, calls `booster.predict(input_data)` and returns the predictions (Rings) in a JSON format.

![Abalone Predictor](../images/byoc-predictor.png)

We use [nginx](https://nginx.org/) as the reverse proxy, [gunicorn](https://gunicorn.org/#deployment) as the web server gateway interface and the inference code as python ["Flask"](https://flask.palletsprojects.com/en/2.3.x/tutorial/factory/) app.

## Dataset and model

For this example, we use a pre-trained [XGBoost](https://xgboost.readthedocs.io) model on [UCI Abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone).

The pre-trained model accepts input in `text/csv` format and returns prediction results in `application/json`

## Prerequisites

Upgrade the below packages to the latest version.

In [ ]:
!pip install -U awscli boto3 sagemaker watermark scikit-learn tqdm --quiet

%load_ext watermark
%watermark -p awscli,boto3,sagemaker,scikit-learn,tqdm

### Inference script for a predictor (XGBoost) model

- In this example, we download a pre-trained XGBoost model on the UCI abalone dataset from s3.
- The inference code is implemented in [`code/inference.py`](./code/inference.py). The [Flask](https://flask.palletsprojects.com/) app implementation is as follows:
  - Implement routes for `/ping` and `/invocations`
  - Implement functions to handle preprocessing, model loading and prediction
  - Predictions will be returned from `/invocations` function

In [ ]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader, s3_path_join
from pathlib import Path

account_id = boto3.client("sts").get_caller_identity().get("Account")
sm_session = session.Session()
region = sm_session._region_name
role = get_execution_role()
bucket = sm_session.default_bucket()

current_dir = os.getcwd()

prefix = "sagemaker/abalone/models/byoc"

predictor_image_name = "abalone/predictor"

pretrained_xgboost_model_s3uri = (
    f"s3://sagemaker-example-files-prod-{region}/models/xgb-abalone/xgboost-model"
)

base_dir = Path("../data").resolve()
predictor_model_dir = Path("./models").absolute()

if not predictor_model_dir.exists():
    predictor_model_dir.mkdir()

In [ ]:
# Download pretrained xgboost model
!aws s3 cp $pretrained_xgboost_model_s3uri $predictor_model_dir

In [ ]:
!pygmentize ./code/inference.py

### Build and test custom inference image locally

 - [Dockerfile](./Dockerfile) implementation
   - Set required LABEL `LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true`
   - Installs required software and python packages
   - Copies all files under code directory to `/opt/program`
   - Sets `ENTRYPOINT` to `["python"]`
   - Sets `CMD` to `["serve"]` (python script that launches nginx, gunicorn in the background)

In [ ]:
!pygmentize Dockerfile

In [ ]:
# build image locally
!docker build -t $predictor_image_name .

### Launch and test custom Inference container locally

In [ ]:
# open a terminal and cd into the predictor directory (the location where predictor.ipynb is located) run the following command
# run this command to launch container locally
# mounts the [models](./models) directory to `/opt/ml/model` directory inside the container and maps container port `8080` to host port `8080`
# docker run --rm -v $(pwd)/models:/opt/ml/model -p 8080:8080 abalone/predictor

#### Check container health by invoking `/ping`

If the `/ping` was successful you should see a response similar to `"GET /ping HTTP/1.1" 200 1` in the terminal

In [ ]:
# Ping local inference endpoint
# !curl http://localhost:8080/ping

### Verify container logs locally (using docker logs)

- To inspect a running container to view container config values or IP address we use `docker inspect <CONTAINER_ID_OR_NAME>`
- To view and tail logs generated in the container we use `docker logs --follow <NUM_OF_LINES> <CONTAINER_ID_OR_NAME>`
- SageMaker publishes container logs to CloudWatch. CloudWatch logs for a given endpoint are published to the following log stream path
`/aws/sagemaker/Endpoints/ENDPOINT_NAME/VARIANT_NAME/CONTAINER_NAME`

**NOTE:** 
1. Run this command in a terminal as running this inside a cell would hang execution.
1. the below command assumes there is only one running container. If you have more, then use command with container name `docker inspect <CONTAINER_ID_OR_NAME>` 

In [ ]:
# RUN THE BELOW IN A SEPARATE NEW TERMINAL
# docker ps --format "{{.Names}}" | xargs -n1 -I{} docker logs --follow --tail 50 {}

### Troubleshooting container locally (using logs)

`!docker logs abalone/featurizer`

#### Test records for inference

Grab a test record from [abalone_test_predictor.csv](../featurizer/data/abalone_test_predictor.csv), generated by [`featurizer.ipynb`](../featurizer/featurizer.ipynb), format it as a CSV record, and send it as raw data to the endpoint `http://localhost:8080/invocations` path

Uncomment below cells to test inference on local container.

In [ ]:
# Send test records to /invocations on the endpoint
# !curl --data-raw '-1.3317586042173168,-1.1425409076053987,-1.0579488602777858,-1.177706547272754,-1.130662184748842,-1.1493955859050584,-1.139968767909096,0.0,1.0,0.0' \
# -H 'Content-Type: text/csv; charset=utf-8' \
# -v http://localhost:8080/invocations

In [ ]:
# Send test records to /invocations on the endpoint
# !curl --data-raw '0.7995425613971686,0.877965470587042,1.326659055767273,1.398563012556441,0.9896192483949702,1.509166873607132,2.01650402614155,0.0,0.0,1.0' \
# -H 'Content-Type: text/csv; charset=utf-8' \
# -v http://localhost:8080/invocations

### Tag and push the local image to private ECR

- Tag the `abalone/predictor` local image to `{account_id}.dkr.ecr.{region}.amazonaws.com/{imagename}:{tag}` format
- Run [./build_n_push.sh](./build_n_push.sh) shell script with image name `nginx` as parameter


In [ ]:
!chmod +x ./build_n_push.sh

!./build_n_push.sh abalone/predictor